In [ ]:
#requires Biopython library
#!pip install biopython
from Bio import SeqIO
import pandas as pd
import regex
import statistics
import os
import glob
#function for counting linkers containing conserved motif between individual zinc-fingers in iTOL-formatted dataset 
# using associated fasta file 
def linker_counting(data,fasta):
    records = list(SeqIO.parse(fasta, "fasta"))
    fastas={'id':[],
           'seq':[]}
    for i in range(len(records)):
        fastas['id'].append(records[i].id.split(sep='|')[1])
        fastas['seq'].append(str(records[i].seq))
    fasta_df=pd.DataFrame(fastas)
    clusters_count=0
    cluster_size=1
    clusters2=0
    clusters3=0
    clusters4=0
    clusters5=0
    clusters6=0
    clusters7=0
    clusters8=0
    clusters9=0
    clusters10=0
    clusters11=0
    clusters12=0
    clusters13=0
    largest_cluster=0
    TGEKP_total=0
    linkers_total=0
    cluster_numbers={
        'total':[],
        '2ZF':[],
        '3ZF':[],
        '4ZF':[],
        '5ZF':[],
        '6ZF':[],
        '7ZF':[],
        '8ZF':[],
        '9ZF':[],
        '10ZF':[],
        '11ZF':[],
        '12ZF':[],
        '>12ZF':[],
        'largest cluster':[],
        'total C2H2 domains':[],
        'median number of C2H2 domains in protein':[],
        'total clusters':[],
        'median cluster size':[],
    }
    linker={
        'total':[],
        '2ZF':[],
        '3ZF':[],
        '4ZF':[],
        '5ZF':[],
        '6ZF':[],
        '7ZF':[],
        '8ZF':[],
        '9ZF':[],
        '10ZF':[],
        '11ZF':[],
        '12ZF':[],
        '>12ZF':[],    
    }
    proteins_total=0
    total_clusters=[]
    C2H2s_in_protein=[]
    C2H2s_total=0
    #The consensus sequence of linker. 
    #i<=2 means no more than two insertions are allowed,
    #s<=2 means no more than two substitutions are allowed
    pattern_string = 'TGEKP'
    r = regex.compile('(%s){i<=2,s<=2}' % pattern_string)
    for index,protein in data.iterrows():
            C2H2=0
            if 'C2H2' in protein[0]: #checking of the presence of additional domains
                proteins_total+=1
                #parsing iTOL dataset
                domains=protein[0].split(sep='\t')
                sequence=list(fasta_df[fasta_df['id']==domains[0].split(sep='|')[0]]['seq'])
                zfs={
                     'start': [],
                     'stop': [],
                       }
                for domain in domains:
                    if 'ZnF_C2H2' in domain:
                        C2H2s_total+=1
                        C2H2+=1
                        zf=domain.split(sep='|')
                        zfs['start'].append(zf[1])
                        zfs['stop'].append(zf[2])
                zfs=pd.DataFrame(zfs)
                C2H2s_in_protein.append(C2H2)
                #searching of the largest cluster in the protein
                clusters=[]
                linkers=[]
                count=0
                link=0
                linker_TGEKP=0
                for znf in range(len(zfs['start'])):
                    try:
                        if (int(zfs['start'][znf+1])-int(zfs['stop'][znf]))<11:
                            count+=1
                            link+=1
                            try:
                                if r.search(str(sequence)[(int(zfs['stop'][znf])+2):(int(zfs['start'][znf+1])+2)]):
                                    linker_TGEKP+=1
                            except:
                                None
                        else:
                            count+=1
                            clusters.append(count)
                            linkers.append(link)
                            if count>=2:
                                total_clusters.append(count)
                            count=0
                    except:
                        count+=1
                        clusters.append(count)
                        if count>=2:
                                total_clusters.append(count)
                max_cluster=max(clusters)
                TGEKP_total+=linker_TGEKP
                linkers_total+=link
                try:
                    max_linkers=max(linkers)
                    linker_ratio=linker_TGEKP/max_linkers
                except:
                    linker_ratio=0
                if max_cluster ==2:
                    cluster_numbers['2ZF'].append(linker_TGEKP)
                elif max_cluster ==3:
                    cluster_numbers['3ZF'].append(linker_TGEKP)
                elif max_cluster ==4:
                    cluster_numbers['4ZF'].append(linker_TGEKP)
                elif max_cluster ==5:
                    cluster_numbers['5ZF'].append(linker_TGEKP)
                elif max_cluster ==6:
                    cluster_numbers['6ZF'].append(linker_TGEKP)
                elif max_cluster ==7:
                    cluster_numbers['7ZF'].append(linker_TGEKP)
                elif max_cluster ==8:
                    cluster_numbers['8ZF'].append(linker_TGEKP)
                elif max_cluster ==9:
                    cluster_numbers['9ZF'].append(linker_TGEKP)
                elif max_cluster ==10:
                    cluster_numbers['10ZF'].append(linker_TGEKP)
                elif max_cluster ==11:
                    cluster_numbers['11ZF'].append(linker_TGEKP)
                elif max_cluster ==12:
                    cluster_numbers['12ZF'].append(linker_TGEKP)
                elif max_cluster >12:
                    cluster_numbers['>12ZF'].append(linker_TGEKP)
                if max_cluster>largest_cluster:
                    largest_cluster=max_cluster
    categories=['2ZF','3ZF','4ZF','5ZF','6ZF','7ZF','8ZF','9ZF','10ZF','11ZF','12ZF','>12ZF']
    linker['total'].append(round((TGEKP_total/proteins_total),2))
    for i in categories:
        try:
            linker[i].append(round(statistics.mean(cluster_numbers[i]),2))
        except:
            linker[i].append(0)
    return linker
#function for counting conserved linkers in tsv files
def linker_counting_tsv(data,fasta):
    records = list(SeqIO.parse(fasta, "fasta"))
    fastas={'id':[],
           'seq':[]}
    for i in range(len(records)):
        fastas['id'].append(records[i].id.split(sep='|')[0])
        fastas['seq'].append(str(records[i].seq))
    fasta_df=pd.DataFrame(fastas)
    clusters_count=0
    cluster_size=1
    clusters2=0
    clusters3=0
    clusters4=0
    clusters5=0
    clusters6=0
    clusters7=0
    clusters8=0
    clusters9=0
    clusters10=0
    clusters11=0
    clusters12=0
    clusters13=0
    largest_cluster=0
    TGEKP_total=0
    linkers_total=0
    cluster_numbers={
        'total':[],
        '2ZF':[],
        '3ZF':[],
        '4ZF':[],
        '5ZF':[],
        '6ZF':[],
        '7ZF':[],
        '8ZF':[],
        '9ZF':[],
        '10ZF':[],
        '11ZF':[],
        '12ZF':[],
        '>12ZF':[],
        'largest cluster':[],
        'total C2H2 domains':[],
        'median number of C2H2 domains in protein':[],
        'total clusters':[],
        'median cluster size':[],
    }
    linker={
        'total':[],
        '2ZF':[],
        '3ZF':[],
        '4ZF':[],
        '5ZF':[],
        '6ZF':[],
        '7ZF':[],
        '8ZF':[],
        '9ZF':[],
        '10ZF':[],
        '11ZF':[],
        '12ZF':[],
        '>12ZF':[],    
    }
    proteins_total=0
    total_clusters=[]
    C2H2s_in_protein=[]
    C2H2s_total=0
    #The consensus sequence of linker. 
    #i<=2 means no more than two insertions are allowed,
    #s<=2 means no more than two substitutions are allowed
    pattern_string = 'TGEKP'
    r = regex.compile('(%s){i<=2,s<=2}' % pattern_string)
    for index,protein in data.iterrows():
        C2H2=0
        proteins_total+=1
        sequence=list(fasta_df[fasta_df['id']==protein['Accession']]['seq'])
        zfs={
             'start': [],
             'stop': [],
               }
        domains = protein['Matches'].split(sep=',')
        for domain in domains:
            C2H2s_total+=1
            C2H2+=1
            zf=domain.split(sep='..')
            zfs['start'].append(zf[0])
            zfs['stop'].append(zf[1])
        zfs=pd.DataFrame(zfs)
        C2H2s_in_protein.append(C2H2)
        clusters=[]
        linkers=[]
        count=0
        link=0
        linker_TGEKP=0
        for znf in range(len(zfs['start'])):
            try:
                if (int(zfs['start'][znf+1])-int(zfs['stop'][znf]))<11:
                    count+=1
                    link+=1
                    try:
                        #the window for motif search should be wider than assigned domain borders
                        if r.search(str(sequence)[(int(zfs['stop'][znf])-3):(int(zfs['start'][znf+1])+4)]):
                            linker_TGEKP+=1
                    except:
                        None
                else:
                    count+=1
                    clusters.append(count)
                    linkers.append(link)
                    if count>=2:
                        total_clusters.append(count)
                    count=0
            except:
                count+=1
                clusters.append(count)
                if count>=2:
                        total_clusters.append(count)
        max_cluster=max(clusters)
        TGEKP_total+=linker_TGEKP
        linkers_total+=link
        try:
            max_linkers=max(linkers)
            linker_ratio=linker_TGEKP/max_linkers
        except:
            linker_ratio=0
        if max_cluster ==2:
            cluster_numbers['2ZF'].append(linker_TGEKP)
        elif max_cluster ==3:
            cluster_numbers['3ZF'].append(linker_TGEKP)
        elif max_cluster ==4:
            cluster_numbers['4ZF'].append(linker_TGEKP)
        elif max_cluster ==5:
            cluster_numbers['5ZF'].append(linker_TGEKP)
        elif max_cluster ==6:
            cluster_numbers['6ZF'].append(linker_TGEKP)
        elif max_cluster ==7:
            cluster_numbers['7ZF'].append(linker_TGEKP)
        elif max_cluster ==8:
            cluster_numbers['8ZF'].append(linker_TGEKP)
        elif max_cluster ==9:
            cluster_numbers['9ZF'].append(linker_TGEKP)
        elif max_cluster ==10:
            cluster_numbers['10ZF'].append(linker_TGEKP)
        elif max_cluster ==11:
            cluster_numbers['11ZF'].append(linker_TGEKP)
        elif max_cluster ==12:
            cluster_numbers['12ZF'].append(linker_TGEKP)
        elif max_cluster >12:
            cluster_numbers['>12ZF'].append(linker_TGEKP)
        if max_cluster>largest_cluster:
            largest_cluster=max_cluster
    categories=['2ZF','3ZF','4ZF','5ZF','6ZF','7ZF','8ZF','9ZF','10ZF','11ZF','12ZF','>12ZF']
    linker['total'].append(round((TGEKP_total/proteins_total),2))
    for i in categories:
        try:
            linker[i].append(round(statistics.mean(cluster_numbers[i]),2))
        except:
            linker[i].append(0)
    return linker
names=[]
clust =[]
for file in glob.glob("*.txt"):
        data =pd.read_csv(file, skiprows=[0,1,2,3,4,5,6,7,8,9,10,11] )
        name=os.path.basename(file)
        base_name=name.split(sep='.')[0]
        fasta=os.path.join(base_name+'.fasta')
        clust.append(linker_counting(data,fasta))
        names.append(base_name)
for file in glob.glob("*.tsv"):
    data =pd.read_table(file)
    name=os.path.basename(file)
    base_name=name.split(sep='.')[0]
    fasta=os.path.join(base_name+'.fasta')
    clust.append(linker_counting_tsv(data,fasta))
    names.append(base_name)
pd.DataFrame(index=names,data=clust)